In [ ]:
TODO : merge with unfittedFEM.ipynb

# CutFEM by ngsxfem

## To solve unfitted interface problem




The PDE and interface/boundary conditions: 

$$
\begin{aligned}
          - \nabla \cdot (\alpha_{\pm} \nabla u) = & \, f 
          & & \text{in}~~ \Omega_{\pm}, 
          \\
          [u]_{\Gamma} = & \, 0 
          &  & \text{on}~~ \Gamma, 
          \\
          [-\alpha \nabla u \cdot \mathbf{n}]_{\Gamma}   = & \, 0 
          &  & \text{on}~~ \Gamma,
          \\
          u = & \, u_D
          &  & \text{on}~~ \partial \Omega.
        \end{aligned}
$$

To find the solution $u$ in two subdomains:

$$
u = \left\{ \begin{array}{cc} u_- & \text{ in } \Omega_-, \\ u_+ & \text{ in } \Omega_+. \end{array} \right.
$$

## Weak formulation of Nitsche's method


To find $u \in V_h^\Gamma := V_h |_{\Omega_+^\text{lin}} \oplus V_h |_{\Omega_-^\text{lin}}$ such that
$$
a(u,v) = f(v) \qquad \forall v \in V_h^\Gamma
$$
where
$$
\begin{aligned}
a(u,v) = & \int_{\Omega_\pm} \alpha_\pm \nabla u \nabla v \,d\omega 
         - \int_{\Gamma} \{ \alpha \nabla u \cdot \mathbf{n} \} [v] \,d\gamma 
         - \int_{\Gamma} \{ \alpha \nabla v \cdot \mathbf{n} \} [u] \,d\gamma
         + \int_{\Gamma} \frac{\lambda}{h} \overline{\alpha} [u] [v] \,d\gamma \\
f(v) = & \int_{\Omega_-} f_-^\text{mf} v \, d\omega
       + \int_{\Omega_+} f_+^\text{mf} v \, d\omega
\end{aligned}
$$

$\{\cdot\}$ denotes the Hansbo-averaging $\{v\}:=\sum_{i=1}^2 \kappa_i v_i$ with the cut ratio $\kappa_i = \frac{|T_i|}{|T|}$; 

$[\cdot]$ denotes the jump across the interface; 

$\lambda$ denotes the stabilization parameter that has to be chosen larger than a constant depending on the shape regularity of $T$. 

## NGSolve Setup

First of all, let's import netgen, ngsolve, and xfem libraries. 

In [1]:
import netgen.gui
%gui tk
import tkinter

# the constant pi
from math import pi
# ngsolve stuff
from ngsolve import *
# visualization stuff
from ngsolve.internal import *
# basic xfem functionality
from xfem import *
from xfem.lsetcurv import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
from netgen.csg import *
# error plot features
import matplotlib.pyplot as plt

## Mesh generation

Second, let's generate a mesh of triangles for a square domain $[-1.5,1.5]^2$ centered on the origin. 

In [2]:
# We generate the background mesh of the domain and use a simplicial triangulation
# To obtain a mesh with quadrilaterals use 'quad_dominated=True'
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.2, quad_dominated=False))
Draw(mesh)

## Manufactured solution

To meassure the error of our numerical solution, we can employ a manufactured solution $u$ such that

$$
u = \left\{ \begin{array}{cc} 1 + \frac{\pi}{2} - \sqrt{2}\cos\frac{\pi}{4}(x^4 + y^4) & \text{ in } \Omega_-, \\ \frac{\pi}{2}(x^4 + y^4)^{\frac{1}{4}} & \text{ in } \Omega_+, \end{array} \right.
$$

with corresponding source term $f_{\pm}^\text{mf}$.

In [3]:
# manufactured solution and corresponding r.h.s. data CoefficientFunctions:
r44 = (x*x*x*x+y*y*y*y)
r41 = sqrt(sqrt(x*x*x*x+y*y*y*y))
r4m3 = (1.0/(r41*r41*r41))
r66 = (x*x*x*x*x*x+y*y*y*y*y*y)
r63 = sqrt(r66)
r22 = (x*x+y*y)
r21 = sqrt(r22)
solution = [1.0+pi/2.0-sqrt(2.0)*cos(pi/4.0*r44),pi/2.0*r41]
coef_f = [ (-1.0*sqrt(2.0)*pi*(pi*cos(pi/4*(r44))*(r66)+3*sin(pi/4*(r44))*(r22))),
          (-2.0*pi*3/2*(r4m3)*(-(r66)/(r44)+(r22))) ]

## Diffusion coefficients

The diffusion coefficients $\alpha_{\pm}$ are

$$
\alpha_{\pm} = \left\{ \begin{array}{cc} 1 & \text{ in } \Omega_-, \\ 
         2 & \text{ in } \Omega_+. \end{array} \right.
$$

In [4]:
# diffusion coefficients for the subdomains (NEG/POS):
alpha = [1.0,2.0]

## Level set function

Let's define a level set function $\phi = (x^4 + y^4)^{\frac{1}{4}} - 1$ to split the background domain and to describe the interface by $\phi = 0$ such that

$$
  \Omega_{-} := \{ \phi < 0 \}, \quad
  \Omega_{+} := \{ \phi > 0 \}, \quad
  \Gamma := \{ \phi = 0 \}.
$$

In [5]:
# level set function of the domain (phi = ||x||_4 - 1) and its interpolation:
levelset = (sqrt(sqrt(x*x*x*x+y*y*y*y)) - 1.0)
Draw(levelset,mesh,"levelset")
visoptions.mminval = 0
visoptions.mmaxval = 0
visoptions.autoscale = 0
visoptions.deformation = 1
Redraw()

## The 1st order approximation

We then approximate the level set function numerically by piecewise linear interpolation, denoted by $\phi_h^\text{lin}$: 

In [6]:
lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
Draw(lsetp1,mesh,"levelset_P1")
visoptions.mminval = 0
visoptions.mmaxval = 0
visoptions.autoscale = 0
visoptions.deformation = 1
Redraw()

We denote the discrete level set function by $\phi_h$, while the discrete subdomains $\Omega_+^\text{lin}$ and $\Omega_-^\text{lin}$. 

## Cut FE space

For the finite element discretization, we use standard background FE Spaces restricted to the subdomains:
$$
V_h^\Gamma = V_h |_{\Omega_+^\text{lin}} \oplus V_h |_{\Omega_-^\text{lin}}
$$

In [7]:
# Background FESpaces (used as CutFESpaces lateron):
Vh = H1(mesh, order = 2, dirichlet = [1,2,3,4])
VhG = FESpace([Vh,Vh])
print("unknowns in background FESpace (2 x standard unknowns): ", VhG.ndof)

unknowns in background FESpace (2 x standard unknowns):  2330


In NGSolve/ngsxfem, we simply take the product space $V_h \times V_h$ but mark the irrelevant dofs using the CutInfo-class. 

## Information about cut elements

To know which elements are cut by the interface $ \Gamma = \{ \phi = 0 \} $, the CutInfo-class is ultilized: 

In [8]:
# Gathering information on cut elements:
#  * domain of (volume/boundary) element:
#    * NEG= only negative level set values
#    * POS= only positive level set values
#    * IF= cut element (negative and positive) level set values
#  * cut ratio:
#    If element is cut this describes the ratio between the measure of part in the negative domain
#    and the measure of the full element.
ci = CutInfo(mesh, lsetp1)

## Free dofs setup

We then find our free degrees of freedoms (unknowns to be solved for) by using GetElementsOfType-class. We can ask for BitArrays corresponding to the differently marked elements: 
  * NEG : True if $\phi < 0$ everywhere on this element, else False
  * POS : True if $\phi > 0$ everywhere on this element, else False
  * IF : True if $\phi = 0$ somewhere on this element, else False  
  * HASNEG: True if $\phi < 0$ somewhere on this element, else False 
  * HASPOS: True if $\phi > 0$ somewhere on this element, else False 

In [9]:
# Overwrite freedofs (degrees of freedoms that should be solved for) of VhG to mark only dofs that
# are involved in the cut problem. Use cut information of ci here:
hasneg = ci.GetElementsOfType(HASNEG)  # <- "hasneg": has (also) negative level set values
haspos = ci.GetElementsOfType(HASPOS)  # <- "haspos": has (also) positive level set values
freedofs = VhG.FreeDofs()
freedofs &= CompoundBitArray([GetDofsOfElements(Vh,hasneg),GetDofsOfElements(Vh,haspos)])

## Discrete normal and mesh size

The normal direction can be obtained from the piecewise linear interpolation of the level set function:
$$
  n_h^\text{lin} = \frac{\nabla \phi_h^\text{lin}}{\Vert \nabla \phi_h^\text{lin} \Vert}. 
$$

And the mesh size $h$ can be obtained by a coefficient function. 

In [10]:
# coefficients / parameters:
n = 1.0/grad(lsetp1).Norm() * grad(lsetp1)
h = specialcf.mesh_size

## Cut ratio and Nitsche parameter

We store the cut ratio data in $\kappa$, and set a parameter $\lambda\frac{\overline{\alpha}}{h}$ for Nitsche's stabilization. 

In [11]:
# the cut ratio extracted from the cutinfo-class
kappa = (CutRatioGF(ci),1.0-CutRatioGF(ci))
# Nitsche stabilization parameter:
stab = 20*(alpha[1]+alpha[0])/h

## Trial and test functions

Let's setup trial and test functions, and the gradient of each component. For the numerical fluxes we choose the Hansbo-averaging where the averages are adjusted to the local cut configuration in order to ensure stability of the Nitsche formulation. 

In [12]:
# expressions of test and trial functions (u and v are tuples):
u = VhG.TrialFunction()
v = VhG.TestFunction()

gradu = [grad(ui) for ui in u]
gradv = [grad(vi) for vi in v]

average_flux_u = sum([- kappa[i] * alpha[i] * gradu[i] * n for i in [0,1]])
average_flux_v = sum([- kappa[i] * alpha[i] * gradv[i] * n for i in [0,1]])

## Numerical integration of CutFEM

To integrate over the subdomains or the interface only with a SymbolicBFI, we have to add a levelset_domain argument which is a dictionary: 

In [13]:
# Integration domains for integration on negative/positive subdomains and on the interface:
# Here, the integration is (geometrically) exact if the "levelset"-argument is a piecewise
# (multi-)linear function. The integration order is chosen according to the arguments in the
# multilinear forms (but can be overwritten with "force_intorder" in the integration domain). If the
# "levelset"-argument is not a (multi-)linear function, you can use the "subdivlvl" argument to add
# additional refinement levels for the geometry approximation. 
lset_neg = { "levelset" : lsetp1, "domain_type" : NEG, "subdivlvl" : 0}
lset_pos = { "levelset" : lsetp1, "domain_type" : POS, "subdivlvl" : 0}
lset_if  = { "levelset" : lsetp1, "domain_type" : IF , "subdivlvl" : 0}

## The bi-/linear forms

For the bilinear form we use a Nitsche formulation which involves averages of the fluxes and jumps of the solution across the interface. 

$$
a(u,v) = \int_{\Omega_\pm} \alpha_\pm \nabla u \nabla v \,d\omega 
       - \int_{\Gamma} \{ \alpha \nabla u \cdot \mathbf{n} \} [v] \,d\gamma 
       - \int_{\Gamma} \{ \alpha \nabla v \cdot \mathbf{n} \} [u] \,d\gamma
       + \int_{\Gamma} \frac{\lambda}{h} \overline{\alpha} [u] [v] \,d\gamma
$$
where $\{\cdot\}$ denotes the Hansbo-averaging, and $[\cdot]$ the jump across the interface. 

Specifically, we first integrate over the subdomains

$$
a(u,v) = \int_{\Omega_-} \alpha_- \nabla u \nabla v \, d\omega 
       + \int_{\Omega_+} \alpha_+ \nabla u \nabla v \, d\omega 
       + \cdots
$$

In [14]:
# bilinear forms:
a = BilinearForm(VhG, symmetric = True)
# l.h.s. domain integrals:
a += SymbolicBFI(levelset_domain = lset_neg, form = alpha[0] * gradu[0] * gradv[0])
a += SymbolicBFI(levelset_domain = lset_pos, form = alpha[1] * gradu[1] * gradv[1])

Then we add the consistency term and its symmetric counterpart into the bilinear form

$$
a(u,v) = \cdots - 
         \left( \int_{\Gamma} \{ \alpha \nabla u \cdot \mathbf{n} \} [v] \, d\gamma 
         + \int_{\Gamma} \{ \alpha \nabla v \cdot \mathbf{n} \} [u] \, d\gamma \right) 
         + \cdots
$$

In [15]:
# Nitsche integrals:
a += SymbolicBFI(levelset_domain = lset_if, form = average_flux_u * (v[0]-v[1])
                                                 + average_flux_v * (u[0]-u[1]))

At last we add the stabilization term into the bilinear form

$$
a(u,v) = \cdots +
         \int_{\Gamma} \frac{\lambda}{h} \overline{\alpha} [u] [v] \, d\gamma
$$

In [16]:
# Nitsche integrals:
a += SymbolicBFI(levelset_domain = lset_if , form = stab * (u[0]-u[1]) * (v[0]-v[1]))

And for the linear form we apply the source term corresponding to our manufactured solution. 

$$
f(v) = \int_{\Omega_-} f_-^\text{mf} v \, d\omega + 
       \int_{\Omega_+} f_+^\text{mf} v \, d\omega
$$

In [17]:
f = LinearForm(VhG)
# r.h.s. domain integrals:
f += SymbolicLFI(levelset_domain = lset_neg, form = coef_f[0] * v[0])
f += SymbolicLFI(levelset_domain = lset_pos, form = coef_f[1] * v[1])

## Assemble

Let's build a GridFunction to store our discrete solution, with designated boundary conditions, then do the assembles. 

In [18]:
# solution vector
gfu = GridFunction(VhG)

# setting domain boundary conditions:
gfu.components[1].Set(solution[1], BND)

# setting up matrix and vector
a.Assemble()
f.Assemble()

## To solve the linear system

We can now solve the problem as a linear system, and obtain our numerical solution stored in the GridFunction. 

In [19]:
# homogenization of boundary data and solution of linear system
def SolveLinearSystem():
    rhs = gfu.vec.CreateVector()
    rhs.data = f.vec - a.mat * gfu.vec
    update = gfu.vec.CreateVector()
    update.data = a.mat.Inverse(freedofs) * rhs
    gfu.vec.data += update
SolveLinearSystem()

Recall that the freedofs only marks relevant dofs. 

## Visualization

Let's plot the exact level set function $\phi$, its interpolation $\phi_h^\text{lin}$, and the numerical solution $u_h$ in $\Omega_-$ and $\Omega_+$ respectively. 

In [20]:
# visualization of (discrete) solution: Wherever (interpolated) level set function is negative
# visualize the first component, where it is positive visualize the second component
u_coef = IfPos(lsetp1, gfu.components[1], gfu.components[0])

# visualize levelset, interpolated levelset and discrete solution:
# (Note that the visualization does not respect the discontinuities. They are smeared out. To see
#  kinks or jumps more clearly increase the subdivision option in the visualization.)
Draw(u_coef,mesh,"u")
visoptions.mminval = 0
visoptions.mmaxval = 0
visoptions.autoscale = 1
visoptions.deformation = 1
Redraw()

## Computation of errors

Let's compute the $L^2$-error of our numerical solution $u_h$ by: 
$$
  \Vert u - u_h \Vert_{L^2}^2 = \int_{\Omega_+ \bigcup \Omega_-} (u - u_h)^2 \, d\omega
$$

In [21]:
def ComputeError():
    # Error coefficients:
    err_sqr_coefs = [(gfu.components[i]-solution[i])*(gfu.components[i]-solution[i]) for i in [0,1] ]

    # Computation of L2 error:
    l2error = sqrt(Integrate( levelset_domain=lset_neg, cf=err_sqr_coefs[0], mesh=mesh, order=2)
              + Integrate( levelset_domain=lset_pos, cf=err_sqr_coefs[1], mesh=mesh, order=2))

    print("L2 error: ",l2error)
ComputeError()

L2 error:  0.018719667217329775


## Geometrically high order accurary
### by SetDeformation-class

To be polished. 

In [22]:
lsetad = LevelSetMeshAdaptation(mesh, order = 3, threshold = 1000, discontinuous_qn = True)
deform = lsetad.CalcDeformation(levelset)

mesh.SetDeformation(deform)
a.Assemble()
f.Assemble()
SolveLinearSystem()
ComputeError()
mesh.UnsetDeformation()

L2 error:  0.0006149664260797689


# Thank you for your attention! 